In [1]:
%run wikidata_functions.ipynb

3


In [2]:
import pandas as pd
import spacy
from spacy.matcher import Matcher 

2023-12-09 03:58:24.425972: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from bs4 import BeautifulSoup

In [4]:
import re
import random

In [5]:
htm = open('text/text1.htm').read()

In [6]:
# https://stackoverflow.com/questions/14694482/converting-html-to-text-with-python
soup = BeautifulSoup(htm)
text = soup.get_text()

In [7]:
text = text.replace("\n", " ")
nlp = spacy.load('en_core_web_md')

doc = nlp(text)
sents = list(doc.sents)
sents = sents[1000:1010]
for i, sent in enumerate(sents):
    print(i, sent)

nlp2 = spacy.load('en_core_web_md')

doc0 = nlp2(str(sents[0]))

for tok in doc0:
  print(tok.text, "...", tok.dep_)

ONLY 1 SUB AND 1 OBJ

In [8]:
def get_entities(doc):

    # dep = map(lambda tok: tok.dep_, doc)
    # subjs, objs = 0, 0
    # for d in dep:
    #     if 'obj' in d:
    #         objs += 1
    #     if 'subj' in d:
    #         subjs += 1

    # if objs + subjs != 2:
    #     raise Exception("More than two entities found.")
    
    
    ## chunk 1
    ent1 = ""
    ent2 = ""
    
    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence
    
    prefix = ""
    modifier = ""

  #############################################################
  
    for tok in doc:
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
        if tok.dep_ != "punct":
      # check: token is a compound word or not
            if tok.dep_ == "compound":
                prefix = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " "+ tok.text
            
            # check: token is a modifier or not
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " "+ tok.text
            
            ## chunk 3
            if tok.dep_.find("subj") == True:
                ent1 = modifier +" "+ prefix + " "+ tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""      
            
            ## chunk 4
            if tok.dep_.find("obj") == True:
                ent2 = modifier +" "+ prefix +" "+ tok.text
            
            ## chunk 5  
            # update variables
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text
  #############################################################

    return [ent1.strip(), ent2.strip()]

In [9]:
def get_relation(doc):
    
  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1",[pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [10]:
p = query_wikipedia("Obama")
links = p.links
# links = []
random.sample(links, len(links)//100)

['2004 Democratic National Convention',
 'Timeline of the Barack Obama presidency (2013)',
 'Capital Gazette shooting',
 "Larry O'Brien",
 'Violence Against Women Act',
 'Birmingham campaign',
 'Timeline of the Barack Obama presidency (2009)',
 'Democratic Party of Virginia',
 'Thaddeus McCotter 2012 presidential campaign',
 'Harris Insights & Analytics',
 'Barack Obama Selma 50th anniversary speech',
 'Timeline of the George H. W. Bush presidency',
 'Black film',
 'Timeline of the Barack Obama presidency (2011)',
 'Lisa Murkowski',
 '2008 United States Senate elections',
 'Bill Gates',
 'List of African-American firsts',
 'University of Chicago Laboratory Schools',
 'Lead Belly',
 'Rick Perry 2012 presidential campaign',
 'Becoming (2020 documentary film)',
 'Federal poverty level',
 '2015 Nobel Peace Prize',
 'Bureau of Labor Statistics']

In [18]:
# list of entities to query

entities = ["Obama"]
articles = []
for e in entities:
    l = search_wikipedia(e)
    # print(l)
    if l:
        articles.append(l[0])

# print(articles)

contents = []
for a in articles:
    try:
        page = query_wikipedia(a)
        contents.append(page.content)
    except:
        print(f"{a} page not found")

x, y = [], []
# a little cleaning
for c in contents:
    i = c.find("== References ==")
    c_ = c[:i]
    c_ = re.sub('==.*==', '', c_) # get rid of wikipedia section headers
    c_ = re.sub('\([^\)]*\)', '', c_) # get rid of everything between parentheses
    

    doc = nlp(c_)
    sents = map(lambda s : str(s).replace("\n", ""), doc.sents)

    for s in sents:
        doc_s = nlp(s)
        ent1, ent2 = get_entities(doc_s)
        rel = get_relation(doc_s)

        try:
            Q1 = get_qid(ent1)
            Q2 = get_qid(ent2)
            P = get_pid_from_str(rel)
        except:
            continue

        # print(Q1, P, Q2)
            
        if check_triple(Q1, P, Q2):
            print("TRUE")
            print(Q1, P, Q2)
            print(ent1, rel, ent2)
            x.append(s)
            y.append(True)
        elif check_sp(Q1, P):
            print("FALSE")
            print(Q1, P, Q2)
            print(ent1, rel, ent2)
            x.append(s)
            y.append(False)
        else:
            print("ELSE")
            print(Q1, P, Q2)
            print(ent1, rel, ent2)
            pass

# return x, y

ELSE
Q76 P5059 Q4909834
Obama served as civil rights lawyer
FALSE
Q76 P19 Q18094
Obama born in Honolulu
FALSE
Q76 P2190 Q19822352
Obama moved to stepfather
ELSE
Q9288 P1366 Q2439
he continued 1980
ELSE
Q87167 P1433 Q186350
manuscript published in Father
FALSE
Q76 P69 Q6581478
Obama graduated from magna cum laude
ELSE
Q311681 P11176 Q7416
who got Margaret Thatcher
ELSE
Q167109 P3254 Q1196074
Jager proposed to him
ELSE
Q887314 P580 Q43959
Bo joined by Sunny
ELSE
Q311681 P1343 Q9174
who described religion
ELSE
Q311681 P3014 Q16970
who were church
ELSE
Q51929447 P9883 Q16970
she was church
ELSE
Q51929517 P7081 Q236
they came to me
ELSE
Q257834 P562 Q257834
 Bank 
ELSE
Q3975 P7113 Q12731
case settled out court
ELSE
Q9288 P101 Q35127
he filed website
ELSE
Q9288 P248 Q17422
he stated same sex marriage
ELSE
Q9288 P460 Q17422
he said same sex marriage
ELSE
Q15614918 P3344 Q2165493
that vetoed pipeline
ELSE
Q10225 P10695 Q1996
Congress introduced 2009
ELSE
Q1379733 P3254 Q851
Obama administratio

In [14]:
# re.sub('\([^\)]*\)', '', contents[0])

In [16]:
x

['Obama was born in Honolulu, Hawaii.',
 'At the age of six, Obama and his mother had moved to Indonesia to join his stepfather.',
 'Obama graduated from Harvard Law in 1991 with a Juris Doctor magna cum laude.',
 "The Barack Obama Presidential Center is Obama's planned presidential library."]

In [17]:
y

[False, False, False, False]

In [105]:
ent1, ent2, rel

('presidential  library', '2021', 'began')

In [110]:
get_qid('48')

[('Q30957', '48', 'year'), ('Q712764', '48', 'natural number'), ('Q8042637', 'XXXY syndrome', 'chromosomal anomaly of the aneuploidic type characterized by the presence of two extra X chromosomes in males'), ('Q1166570', '48', 'Wikimedia disambiguation page'), ('Q16926797', 'route 48', 'tram route in metropolitan Melbourne, Victoria, Australia')]


'Q30957'

In [104]:
s = 'His presidential library in Chicago began construction in 2021'
doc = nlp(s)
ent1, ent2 = get_entities(doc)
rel = get_relation(doc)

Q1 = get_qid(ent1)
Q2 = get_qid(ent2)
try:
    P = get_pid_from_str(rel)
except:
    pass

[]


IndexError: list index out of range

In [87]:
Q1

'Q146'

In [51]:
doc = nlp(s)

In [52]:
for tok in doc:
  print(tok.text, "...", tok.dep_)

Barack ... compound
Hussein ... nsubj
Obama ... compound
II ... appos
( ... punct
  ... dep
bə ... amod
- ... punct
RAHK ... amod
hoo ... amod
- ... punct
SAYN ... nmod
oh ... compound
- ... punct
BAH ... compound
- ... punct
mə ... parataxis
; ... punct
born ... advcl
August ... npadvmod
4 ... nummod
, ... punct
1961 ... nummod
) ... punct
is ... ROOT
an ... det
American ... amod
politician ... attr
who ... nsubj
served ... relcl
as ... prep
the ... det
44th ... amod
president ... pobj
of ... prep
the ... det
United ... compound
States ... pobj
from ... prep
2009 ... pobj
to ... prep
2017 ... pobj
. ... punct
